# Prevendo o preço de Automóveis

In [37]:
import pandas as pd
base = pd.read_csv("./autos.csv", encoding="ISO-8859-1")
base.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


# Removendo colunas que não agregam na base de dados

 Isso é feito afim de reduzir a base de dados para os atributos mais relevantes, assim, a o algorítimo terá apenas dados relevantes sobre o problema em questão

In [38]:
base = base.drop("dateCrawled", axis=1)
base = base.drop("nrOfPictures", axis=1)
base = base.drop("dateCreated", axis=1)
base = base.drop("postalCode", axis=1)
base = base.drop("lastSeen", axis=1)

base.shape

(371528, 15)

In [39]:
base["name"].value_counts()


Ford_Fiesta                                                          657
BMW_318i                                                             627
Opel_Corsa                                                           622
Volkswagen_Golf_1.4                                                  603
BMW_316i                                                             523
                                                                    ... 
Unikatischer_Opel_Astra                                                1
Golf_VII_Comfortline_BMT_1_6l_TDI_/_Garantie_/_nur_34000_km_/_Top      1
Volkswagen_Touran_1.9_TDI_Conceptline__AHK                             1
Audi_tt_8n_Grill                                                       1
Golf_4____Top_Zustand                                                  1
Name: name, Length: 233531, dtype: int64

In [40]:
base = base.drop("name", axis=1)

In [41]:
base["seller"].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [42]:
base = base.drop("seller", axis=1)

In [43]:
base["offerType"].value_counts()

Angebot    371516
Gesuch         12
Name: offerType, dtype: int64

In [44]:
base = base.drop("offerType", axis=1)

In [45]:
base.shape

(371528, 12)

# Pré-Processamento de valores inconsistente

In [46]:
i1 = base.loc[base.price <= 10]

In [47]:
i1.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,1,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,4,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,12,benzin,volkswagen,NaN


In [48]:
base = base[base.price > 10]

In [49]:
base.shape

(359410, 12)

In [50]:
i2 = base.loc[base.price > 350000]

In [51]:
i2.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein


In [52]:
base = base.loc[base.price < 350000]

In [53]:
base.shape

(359291, 12)

# Pré-processamento de valores faltantes

In [54]:
# Verificando os valores faltantes do atributo vehicleType para que posteriormente no código seja modificados

base[pd.isnull(base["vehicleType"])]


,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [55]:
# Verificando qual o vehicleType mais se repete na base de dados, afim de subistituir os valores faltantes desse
# atibuto

base["vehicleType"].value_counts() # limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [56]:
# Verificando os valores faltantes desse atribulto para que eles sejam subistituidos

base.loc[pd.isnull(base["gearbox"])]


,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [57]:
# verificando qual o gearbox mais aparece na base de dados, para que posteriormente possa ser utilizada NaN
# desse atribulto

base["gearbox"].value_counts() # manuell


manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [58]:
# Verificando os valores faltantes desse atribulto para que eles sejam subistituidos

base.loc[pd.isnull(base["model"])]


,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [59]:
# verificando qual o model mais aparece na base de dados, para que posteriormente possa ser utilizada NaN
# desse atribulto

base["model"].value_counts() # golf


golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
discovery_sport        1
serie_1                1
Name: model, Length: 251, dtype: int64

In [60]:
# Verificando os valores faltantes desse atribulto para que eles sejam subistituidos

base.loc[pd.isnull(base["fuelType"])]


,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [61]:
# verificando qual o fuelType mais aparece na base de dados, para que posteriormente possa ser utilizada NaN
# desse atribulto

base["fuelType"].value_counts() # benzin


benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [62]:
# Verificando os valores faltantes desse atribulto para que eles sejam subistituidos

base.loc[pd.isnull(base["notRepairedDamage"])]


,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [63]:
# verificando qual o notRepairedDamage mais aparece na base de dados, para que posteriormente possa ser utilizada NaN
# desse atribulto

base["notRepairedDamage"].value_counts() # nein


nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

# Substituindo os valores faltantes da base de dados

In [64]:
valores = {"vehicleType": "limousine", "gearbox": "manuell",
           "model": "golf", "fuelType": "benzin", "notRepairedDamage": "nein"}
print(valores)


{'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}


   ### Subistituindo os valores NaN das colunas citadas no Dicionário assina, pelas chaves do dict

In [65]:
base = base.fillna(value=valores)

# Separando os atributos Previsores e da Classe

In [66]:
# Separando os previsores em um array numpy

previsores = base.iloc[:, 1:13].values

In [67]:
# Separando a classe em um array numpy

preco_real = base.iloc[:, 0].values

# Aplicando o LabelEncoder e o One Hot Encoder
É uma Classe do pandas que transforma dados categóricos(palavras) em dados numéricos, isso é feito, pois, modelos de Machile Learning só conseguem utilizar valores numéricos para realizar os cálculos

In [68]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_previsores = LabelEncoder()
onehotencorder = ColumnTransformer(transformers=[("OneHot", 
                                                  OneHotEncoder(), 
                                                  [0,1,3,5,8,9,10])],
                                                   remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()


In [69]:
previsores.shape

(359291, 316)

# Criar a Estrutura do modelo de Regressão

In [71]:
# Importando as libs

from keras.models import Sequential
from keras.layers import Dense

# Criando o modelo
regressor = Sequential()
regressor.add(Dense(units=158, activation="relu", input_dim=316))
regressor.add(Dense(units=158, activation="relu"))
regressor.add(Dense(units=1, activation="linear"))

# Compilando o modelo criado
regressor.compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])


In [72]:
# Treinando o modelo
regressor.fit(previsores, preco_real, batch_size=300, epochs=50)

Epoch 1/50
1198/1198 [==============================] - 21s 13ms/step - loss: 4318.1617 - mean_absolute_error: 4318.1617
Epoch 2/50
1198/1198 [==============================] - 17s 15ms/step - loss: 3405.5071 - mean_absolute_error: 3405.5071
Epoch 3/50
1198/1198 [==============================] - 17s 14ms/step - loss: 3110.4522 - mean_absolute_error: 3110.4522
Epoch 4/50
1198/1198 [==============================] - 17s 14ms/step - loss: 2951.9417 - mean_absolute_error: 2951.9417
Epoch 5/50
1198/1198 [==============================] - 17s 14ms/step - loss: 2840.3579 - mean_absolute_error: 2840.3579
Epoch 6/50
1198/1198 [==============================] - 10s 8ms/step - loss: 2773.9577 - mean_absolute_error: 2773.9576
Epoch 7/50
1198/1198 [==============================] - 9s 8ms/step - loss: 2719.2023 - mean_absolute_error: 2719.2023
Epoch 8/50
1198/1198 [==============================] - 9s 7ms/step - loss: 2657.2239 - mean_absolute_error: 2657.2239
Epoch 9/50
1198/1198 [===============